In [1]:
import math
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.cluster import KMeans
print("setup done")

setup done


In [2]:
df = pd.read_csv('./dataset/brazil_report.csv', sep=';', parse_dates=['Sale Date Time'])
del df['Employee']
df['Date'] = pd.to_datetime(df['Sale Date Time'])
df = df.drop(columns=['Sale Date Time'])
df = df.loc[(df['Client'] != 'Customer not informed')]
print("dataset Loaded")

dataset Loaded


In [3]:
df.sort_values(by='Order Number').head()

,Company Code,Order Number,Product,Product Category,Client,Client City,Product Cost,Discount Amount,Amount,Total,Form of payment,Date
2272,1420000,29,Gasoline,Fuel,Gerald Reeves,Agririe,3.043,0.0,13.893,50.00,Money,2017-04-01 08:03:00
7,39000,50,Special Diesel,Fuel,Arthur Mason,Froit,2.510,0.0,17.864,50.00,Money,2017-03-31 04:35:00
18,39000,88,Special Diesel,Fuel,Laverne Logan,Agririe,2.510,0.0,70.729,197.97,Debit Card,2017-03-31 05:45:00
19,39000,88,Lubricant 2658,Lubricant,Laverne Logan,Agririe,6.738,0.0,1.000,11.00,Debit Card,2017-03-31 05:45:00
94711,367000,93,Special Gasoline,Fuel,Barry Barrett,Ruwell,3.050,0.0,27.863,100.00,Money,2017-04-20 08:40:00


In [4]:
df.describe()

,Company Code,Order Number,Product Cost,Discount Amount,Amount,Total
count,1.628925e+06,1.628925e+06,1.628925e+06,1.628925e+06,1.628925e+06,1.628925e+06
mean,3.385974e+06,1.014284e+06,3.753446e+00,5.306870e-02,4.359097e+01,1.508061e+02
std,3.638789e+06,6.521031e+05,5.483155e+00,2.582094e+00,7.629997e+01,2.433236e+02
min,3.900000e+04,2.900000e+01,-1.794000e+00,0.000000e+00,3.000000e-03,1.000000e-02
25%,8.820000e+05,4.861900e+05,2.966000e+00,0.000000e+00,1.219900e+01,5.000000e+01
50%,1.966000e+06,8.602610e+05,3.399000e+00,0.000000e+00,2.538800e+01,1.000000e+02
75%,4.169000e+06,1.540326e+06,3.626000e+00,0.000000e+00,4.436000e+01,1.634500e+02
max,1.460200e+07,2.391974e+06,3.587550e+02,8.034000e+02,3.257799e+03,9.999000e+03


In [5]:
df.loc[(df['Product Cost'] <0)]

,Company Code,Order Number,Product,Product Category,Client,Client City,Product Cost,Discount Amount,Amount,Total,Form of payment,Date
26179990,39000,2336301,Chewing Gum And Candy 5359,Chewing Gum And Candy,Shelley Flores,Agririe,-1.794,0.0,1.0,3.5,Extended payment plan/ In installments,2020-03-04 13:05:00


In [6]:
len(df)
# cleaning done

1628925

In [7]:
df["Form of payment"].unique()

array(['Money', 'Debit Card', 'Credit Card',
       'Extended payment plan/ In installments',
       'Pre-paid check / Pre- date check', 'Check', 'Others'],
      dtype=object)

In [8]:
# len(df.loc[(df['Client'] != 'Customer not informed') & (df['Client City'] != 'No City')])
# # df.loc[(df['Client City'] == 'No City') ]

In [9]:
df.isnull().values.any()

False

In [10]:
df.describe()

,Company Code,Order Number,Product Cost,Discount Amount,Amount,Total
count,1.628925e+06,1.628925e+06,1.628925e+06,1.628925e+06,1.628925e+06,1.628925e+06
mean,3.385974e+06,1.014284e+06,3.753446e+00,5.306870e-02,4.359097e+01,1.508061e+02
std,3.638789e+06,6.521031e+05,5.483155e+00,2.582094e+00,7.629997e+01,2.433236e+02
min,3.900000e+04,2.900000e+01,-1.794000e+00,0.000000e+00,3.000000e-03,1.000000e-02
25%,8.820000e+05,4.861900e+05,2.966000e+00,0.000000e+00,1.219900e+01,5.000000e+01
50%,1.966000e+06,8.602610e+05,3.399000e+00,0.000000e+00,2.538800e+01,1.000000e+02
75%,4.169000e+06,1.540326e+06,3.626000e+00,0.000000e+00,4.436000e+01,1.634500e+02
max,1.460200e+07,2.391974e+06,3.587550e+02,8.034000e+02,3.257799e+03,9.999000e+03


In [11]:
# print(len(df['Client'].unique()))
# print(len(df['Client']))
print(len(df['Order Number']))
print(len(df['Client'].unique()))

1628925
10816


In [12]:


# fig, ax = plt.subplots(figsize=(10,8))
# ax.scatter(df['Amount'], df['Product Cost'])
# ax.set_xlabel('Amount')
# ax.set_ylabel('Product Cost')
# plt.show()



In [13]:
# df['Hour']=df['Date'].dt.hour
# df['Month']=df['Date'].dt.month
# df['Weekdays']= df['Date'].dt.weekday

In [14]:
#function for ordering cluster numbers
def order_cluster(cluster_field_name, target_field_name,df,ascending):
    df_new = df.groupby(cluster_field_name)[target_field_name].mean().reset_index()
    df_new = df_new.sort_values(by=target_field_name,ascending=ascending).reset_index(drop=True)
    df_new['index'] = df_new.index
    df_final = pd.merge(df,df_new[[cluster_field_name,'index']], on=cluster_field_name)
    df_final = df_final.drop([cluster_field_name],axis=1)
    df_final = df_final.rename(columns={"index":cluster_field_name})
    return df_final

In [15]:
dfclient = pd.DataFrame(df['Client'].unique())
dfclient.columns = ['Client']


df_max_purchase = df.groupby('Client').Date.max().reset_index()
df_max_purchase.columns = ['Client','MaxPurchaseDate']
df_max_purchase['Recency'] = (df_max_purchase['MaxPurchaseDate'].max() - df_max_purchase['MaxPurchaseDate']).dt.days
dfclient = pd.merge(dfclient, df_max_purchase[['Client','Recency']], on='Client')

#build 4 clusters for recency and add it to dataframe
kmeans = KMeans(n_clusters=4)
kmeans.fit(dfclient[['Recency']])
dfclient['RecencyCluster'] = kmeans.predict(dfclient[['Recency']])   
dfclient = order_cluster('RecencyCluster', 'Recency',dfclient,False)

dfclient.head(100)

,Client,Recency,RecencyCluster
0,Arthur Mason,0,3
1,Rickey Castillo,1,3
2,Bennie Chavez,3,3
3,Leona Harrison,3,3
4,Lois Anderson,2,3
...,...,...,...
95,Sherman Sherman,22,3
96,Shelia Norman,21,3
97,Theodore Hudson,1,3
98,Troy Price,27,3


In [16]:
dfclient['RecencyCluster'].value_counts()

3    5420
1    2125
2    1777
0    1494
Name: RecencyCluster, dtype: int64

In [17]:
dfclient['Recency'].value_counts()

0       408
1       329
2       220
15      219
3       176
       ... 
403       1
347       1
179       1
147       1
1095      1
Name: Recency, Length: 1020, dtype: int64

In [18]:
#FREQUENCY
                
#get order counts for each user and create a dataframe with it
dffreq = df.groupby('Client')['Order Number'].nunique().reset_index()
dffreq.columns = ['Client','Frequency']
                
#add this data to our main dataframe
dfclient = pd.merge(dfclient, dffreq, on='Client')

#k-means
kmeans = KMeans(n_clusters=4)
kmeans.fit(dfclient[['Frequency']])
dfclient['FrequencyCluster'] = kmeans.predict(dfclient[['Frequency']])

#order the frequency cluster
dfclient = order_cluster('FrequencyCluster', 'Frequency',dfclient,True)

In [19]:
dffreq

,Client,Frequency
0,Aaron Cruz,3
1,Aaron Hampton,91
2,Aaron Howell,50
3,Aaron Nguyen,5
4,Aaron Washington,4
...,...,...
10811,Zachary Santiago,3
10812,Zachary Simmons,160
10813,Zachary Stephens,28
10814,Zachary Stewart,7


In [20]:
len(df.loc[(df["Client"]=='Arthur Mason')]['Order Number'].unique())

34652

In [21]:
dfclient.head(100)

,Client,Recency,RecencyCluster,Frequency,FrequencyCluster
0,Arthur Mason,0,3,34652,2
1,Sonia Mendez,0,3,17039,2
2,Joann Underwood,0,3,14712,2
3,Constance Maldonado,386,2,18533,2
4,Rickey Castillo,1,3,282,0
...,...,...,...,...,...
95,Theodore Hudson,1,3,476,0
96,Troy Price,27,3,108,0
97,Minnie Walton,2,3,339,0
98,Jean Walker,0,3,497,0


In [22]:
# dfclient.sort_values(by='Frequency')

In [23]:
# dfclient.loc[(dfclient['FrequencyCluster']==0)].describe()

In [24]:
# from sklearn.cluster import DBSCAN
# #REVENUE
# #calculate revenue for each customer
# dfrev = df.groupby('Client').Total.sum().reset_index()

# #merge it with our main dataframe
# dfclient = pd.merge(dfclient, dfrev, on='Client')
# #apply clustering
# kmeans = DBSCAN(eps=3, min_samples=2)
# dfclient['RevenueCluster'] = kmeans.fit_predict(dfclient[['Total']])
# # dfclient['RevenueCluster'] = kmeans.predict(dfclient[['Total']])
# #order the cluster numbers
# dfclient = order_cluster('RevenueCluster', 'Total',dfclient,True)

# dfclient.head(100)

In [25]:
#REVENUE
#calculate revenue for each customer
dfrev = df.groupby('Client').Total.sum().reset_index()

#merge it with our main dataframe
dfclient = pd.merge(dfclient, dfrev, on='Client')
#apply clustering
kmeans = KMeans(n_clusters=4)
kmeans.fit(dfclient[['Total']])
dfclient['RevenueCluster'] = kmeans.predict(dfclient[['Total']])
#order the cluster numbers
dfclient = order_cluster('RevenueCluster', 'Total',dfclient,True)

dfclient.head(100)

,Client,Recency,RecencyCluster,Frequency,FrequencyCluster,Total,RevenueCluster
0,Arthur Mason,0,3,34652,2,1.335526e+07,3
1,Barry Barrett,8,3,113604,3,1.553030e+07,3
2,Sonia Mendez,0,3,17039,2,1.869099e+06,1
3,Constance Maldonado,386,2,18533,2,1.832098e+06,1
4,Eloise Morales,0,3,1252,0,5.280048e+05,1
...,...,...,...,...,...,...,...
95,Ivan Fletcher,6,3,985,0,1.319708e+05,0
96,Joanne Jenkins,3,3,111,0,1.363338e+04,0
97,Estelle Craig,19,3,128,0,2.273092e+04,0
98,Wendy Delgado,0,3,199,0,3.282157e+04,0


In [26]:
dfclient['RevenueCluster'].value_counts()

0    10760
1       50
2        4
3        2
Name: RevenueCluster, dtype: int64

In [27]:
dfclient.loc